The idea with this is just to try and see if there is any thing special with null values in indexes.
I thought i understood it as DB viewing null and like this has no value and therefor if you indexed on a column that is nullable then all the null values would not be index and if you searched for *WHERE name IS NULL* then i thought that it would be a full table scan.
But this might not be the case so i will try to figure that out

In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(20000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

So i will try to set some of the fields to null to see what the explain says

In [ ]:
query = """
UPDATE products
SET price = NULL
WHERE id <= 1000
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE price IS NULL;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE price = 100;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price IS NULL;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

query = """
SELECT * FROM products WHERE price = 100;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

So this seems to me to indicate the null values does get stored in the index

It was because i have read the articles on use the index luke where he talks about the ORACLE DB storign nullable columns as partiel indexes or something. But taht is not the case with mysql